# Creating data for pareto

## Load libraries and merge the df

In [1]:
import pandas as pd
import os
import shutil
path = os.chdir('C:/Users/jewel.espiritu/OneDrive - AMDATEX LAS PINAS SERVICES INC/Python/py') 
sladf = pd.concat(pd.read_excel('./data/WeeklySLA_Data_2022.xlsx', sheet_name=None), ignore_index=True)

## Create Pivot table

In [2]:
SLApivot = pd.pivot_table(sladf, index=['Service', 'Olderthan14', 'Week','TTR Result\n(Excluding Pending Time)'], columns=['Operational Status'], values=['Ticket No'], aggfunc='count')
paretopvt = SLApivot.query('Olderthan14=="Y"')
paretopvt

Ticket No  \
Operational Status                                                                                        On-going   
Service                                            Olderthan14 Week TTR Result\n(Excluding Pending Time)             
3rd Party/ Client Controlled Software, Network ... Y           1.0  Beyond TAT                                50.0   
                                                                    Within TAT                                10.0   
                                                               2.0  Beyond TAT                                45.0   
                                                                    Within TAT                                10.0   
                                                               3.0  Beyond TAT                                44.0   
...                                                                                                            ...   
WFH PC setup                                       Y           21.0 Within TAT                                18.0   
                                                               22.0 Beyond TAT                                19.0   
                                                                    Within TAT                                 4.0   
                                                               23.0 Beyond TAT                                 2.0   
                                                                    Within TAT                                 8.0   

                                                                                                                  \
Operational Status                                                                                       Pending   
Service                                            Olderthan14 Week TTR Result\n(Excluding Pending Time)           
3rd Party/ Client Controlled Software, Network ... Y           1.0  Beyond TAT                              11.0   
                                                                    Within TAT                              62.0   
                                                               2.0  Beyond TAT                               9.0   
                                                                    Within TAT                              36.0   
                                                               3.0  Beyond TAT                              12.0   
...                                                                                                          ...   
WFH PC setup                                       Y           21.0 Within TAT                               5.0   
                                                               22.0 Beyond TAT                               NaN   
                                                                    Within TAT                               9.0   
                                                               23.0 Beyond TAT                               NaN   
                                                                    Within TAT                              10.0   

                                                                                                                           
Operational Status                                                                                       Resolved/ Closed  
Service                                            Olderthan14 Week TTR Result\n(Excluding Pending Time)                   
3rd Party/ Client Controlled Software, Network ... Y           1.0  Beyond TAT                                        NaN  
                                                                    Within TAT                                        NaN  
                                                               2.0  Beyond TAT                                        NaN  
                                                                    Within TAT     

## Drop other columns

In [3]:
paretopvt.columns = paretopvt.columns.droplevel(0)
paretopvt.columns.name = None
newdf = paretopvt.reset_index()
newdf

,Service,Olderthan14,Week,TTR Result\n(Excluding Pending Time),On-going,Pending,Resolved/ Closed
0,"3rd Party/ Client Controlled Software, Network...",Y,1.0,Beyond TAT,50.0,11.0,NaN
1,"3rd Party/ Client Controlled Software, Network...",Y,1.0,Within TAT,10.0,62.0,NaN
2,"3rd Party/ Client Controlled Software, Network...",Y,2.0,Beyond TAT,45.0,9.0,NaN
3,"3rd Party/ Client Controlled Software, Network...",Y,2.0,Within TAT,10.0,36.0,NaN
4,"3rd Party/ Client Controlled Software, Network...",Y,3.0,Beyond TAT,44.0,12.0,NaN
...,...,...,...,...,...,...,...
582,WFH PC setup,Y,21.0,Within TAT,18.0,5.0,5.0
583,WFH PC setup,Y,22.0,Beyond TAT,19.0,NaN,8.0
584,WFH PC setup,Y,22.0,Within TAT,4.0,9.0,6.0
585,WFH PC setup,Y,23.0,Beyond TAT,2.0,NaN,17.0


## Turn NaN values into 0 and Remove decimal using lambda

In [4]:
newdf['On-going'] = newdf['On-going'].fillna(0)
newdf['Pending'] = newdf['Pending'].fillna(0)
newdf['Resolved/ Closed'] = newdf['Resolved/ Closed'].fillna(0)

newdf['On-going'] = newdf['On-going'].astype(str).apply(lambda x: x.replace('.0',''))
newdf['Pending'] = newdf['Pending'].astype(str).apply(lambda x: x.replace('.0',''))
newdf['Resolved/ Closed'] = newdf['Resolved/ Closed'].astype(str).apply(lambda x: x.replace('.0',''))
newdf

,Service,Olderthan14,Week,TTR Result\n(Excluding Pending Time),On-going,Pending,Resolved/ Closed
0,"3rd Party/ Client Controlled Software, Network...",Y,1.0,Beyond TAT,50,11,0
1,"3rd Party/ Client Controlled Software, Network...",Y,1.0,Within TAT,10,62,0
2,"3rd Party/ Client Controlled Software, Network...",Y,2.0,Beyond TAT,45,9,0
3,"3rd Party/ Client Controlled Software, Network...",Y,2.0,Within TAT,10,36,0
4,"3rd Party/ Client Controlled Software, Network...",Y,3.0,Beyond TAT,44,12,0
...,...,...,...,...,...,...,...
582,WFH PC setup,Y,21.0,Within TAT,18,5,5
583,WFH PC setup,Y,22.0,Beyond TAT,19,0,8
584,WFH PC setup,Y,22.0,Within TAT,4,9,6
585,WFH PC setup,Y,23.0,Beyond TAT,2,0,17


## Filter the current week

In [5]:
newdf = newdf.query('Week == 23')
newdf = newdf.rename( columns = {'TTR Result\n(Excluding Pending Time)': 'TTR'}, inplace = False)
nnewdf = newdf.query('TTR == "Beyond TAT"')
nnewdf

,Service,Olderthan14,Week,TTR,On-going,Pending,Resolved/ Closed
38,"3rd Party/ Client Controlled Software, Network...",Y,23.0,Beyond TAT,4,11,57
78,Access,Y,23.0,Beyond TAT,7,19,77
118,Email,Y,23.0,Beyond TAT,3,12,21
158,Hardware,Y,23.0,Beyond TAT,6,8,72
197,"IT software assessment, evaluation and testing",Y,23.0,Beyond TAT,0,0,3
236,IT software transferring,Y,23.0,Beyond TAT,0,0,1
261,MAJOR Incident (BSD System/Application),Y,23.0,Beyond TAT,0,0,2
320,Network or internet,Y,23.0,Beyond TAT,2,0,5
360,Onboarding Employee's IT-related Access and Ha...,Y,23.0,Beyond TAT,0,1,13
410,Resignee's Clearance Processing,Y,23.0,Beyond TAT,1,1,2


## Add percentage column for beyond tat

In [6]:
sumongoingtat = nnewdf['On-going'].astype(int).sum()
ongoingpercent = nnewdf['On-going'].astype(int)/sumongoingtat
nnewdf["Ongoing TAT Percent"] = ongoingpercent

C:\Users\jewel.espiritu\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [7]:
nnewdf = nnewdf.rename( columns = {'On-going': 'On_going'}, inplace = False)
nnewdf.On_going = nnewdf.On_going.astype(float)
beyond = "On_going"
nnewdf = nnewdf.sort_values(beyond, ascending=False)

In [8]:
nnewdf['On_going'] = nnewdf['On_going'].astype(str).apply(lambda x: x.replace('.0',''))
nnewdf['cumulative_sum'] = nnewdf[beyond].astype(int).cumsum()
nnewdf['cumulative_perc'] = 100*nnewdf['cumulative_sum']/nnewdf[beyond].astype(int).sum()
nnewdf.to_excel(r'C:\Users\jewel.espiritu\OneDrive - AMDATEX LAS PINAS SERVICES INC\SLA\repo\pareto.xlsx', index = False, header = True)
nnewdf

,Service,Olderthan14,Week,TTR,On_going,Pending,Resolved/ Closed,Ongoing TAT Percent,cumulative_sum,cumulative_perc
509,Software,Y,23.0,Beyond TAT,11,11,65,0.305556,11,30.555556
78,Access,Y,23.0,Beyond TAT,7,19,77,0.194444,18,50.000000
158,Hardware,Y,23.0,Beyond TAT,6,8,72,0.166667,24,66.666667
38,"3rd Party/ Client Controlled Software, Network...",Y,23.0,Beyond TAT,4,11,57,0.111111,28,77.777778
118,Email,Y,23.0,Beyond TAT,3,12,21,0.083333,31,86.111111
320,Network or internet,Y,23.0,Beyond TAT,2,0,5,0.055556,33,91.666667
585,WFH PC setup,Y,23.0,Beyond TAT,2,0,17,0.055556,35,97.222222
410,Resignee's Clearance Processing,Y,23.0,Beyond TAT,1,1,2,0.027778,36,100.000000
197,"IT software assessment, evaluation and testing",Y,23.0,Beyond TAT,0,0,3,0.000000,36,100.000000
236,IT software transferring,Y,23.0,Beyond TAT,0,0,1,0.000000,36,100.000000


## Save a copy to local then send a copy to onedrive repo

In [9]:
FilePath = r'C:\repo\SLA_Pareto.xlsx'
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter(FilePath, engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
nnewdf.to_excel(writer, sheet_name='Pareto', index = False)
writer.save()



'C:\\Users\\jewel.espiritu\\OneDrive - AMDATEX LAS PINAS SERVICES INC\\SLA\\repo\\SLA.xlsx'